In [194]:
from socceraction.data.wyscout import PublicWyscoutLoader
from socceraction.spadl.wyscout import convert_to_actions
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.vaep.labels as lab
import socceraction.vaep.features as fs
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [195]:
api = PublicWyscoutLoader(root="data/wyscout")

In [196]:
# Competitions Input
competitions_df = api.competitions()
competitions_df

,competition_id,season_id,country_name,competition_name,competition_gender,season_name
0,524,181248,Italy,Italian first division,male,2017/2018
1,364,181150,England,English first division,male,2017/2018
2,795,181144,Spain,Spanish first division,male,2017/2018
3,412,181189,France,French first division,male,2017/2018
4,426,181137,Germany,German first division,male,2017/2018
5,102,9291,International,European Championship,male,2016
6,28,10078,International,World Cup,male,2018


In [197]:
# Games Input (Competition_id, Season_id)
games_wc_2018_df = api.games(28, 10078)
games_wc_2018_df

,game_id,competition_id,season_id,game_date,game_day,home_team_id,away_team_id
0,2058017,28,10078,2018-07-15 15:00:00,0,4418,9598
1,2058016,28,10078,2018-07-14 14:00:00,0,5629,2413
2,2058015,28,10078,2018-07-11 18:00:00,0,9598,2413
3,2058014,28,10078,2018-07-10 18:00:00,0,4418,5629
4,2058012,28,10078,2018-07-07 18:00:00,0,14358,9598
...,...,...,...,...,...,...,...
59,2057966,28,10078,2018-06-16 10:00:00,1,4418,8493
60,2057960,28,10078,2018-06-15 18:00:00,1,9905,1598
61,2057961,28,10078,2018-06-15 15:00:00,1,16216,10840
62,2057955,28,10078,2018-06-15 12:00:00,1,16129,15670


In [198]:
# Teams Input (game_id)
teams_wc_2018_df = api.teams(2058017)
teams_wc_2018_df.head()

,team_id,team_name_short,team_name
0,9598,Croatia,Croatia
1,4418,France,France


In [199]:
# Events Input (game_id)
event_wc_2018_df = api.events(2058017)
event_wc_2018_df['minutes_played'] = event_wc_2018_df['milliseconds'] / 60000
event_wc_2018_df

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags,minutes_played
0,263883958,2058017,1,1892.339,9598,14943,8,Pass,85,Simple pass,"[{'y': 51, 'x': 50}]",[],0.031539
1,263883959,2058017,1,3889.375,9598,69968,8,Pass,85,Simple pass,"[{'y': 52, 'x': 39}, {'y': 74, 'x': 34}]",[{'id': 1801}],0.064823
2,263883960,2058017,1,6140.946,9598,8287,8,Pass,85,Simple pass,"[{'y': 74, 'x': 34}, {'y': 93, 'x': 30}]",[{'id': 1801}],0.102349
3,263883963,2058017,1,9226.570,9598,69409,8,Pass,85,Simple pass,"[{'y': 93, 'x': 30}, {'y': 65, 'x': 11}]",[{'id': 1801}],0.153776
4,263883964,2058017,1,12658.969,9598,135747,8,Pass,84,Launch,"[{'y': 65, 'x': 11}, {'y': 51, 'x': 64}]",[{'id': 1801}],0.210983
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,263885652,2058017,2,2978301.867,9598,3476,8,Pass,85,Simple pass,"[{'y': 20, 'x': 46}, {'y': 6, 'x': 64}]",[{'id': 1801}],49.638364
1458,263885653,2058017,2,2979084.611,9598,14812,7,Others on the ball,72,Touch,"[{'y': 6, 'x': 64}, {'y': 2, 'x': 82}]",[],49.651410
1459,263885654,2058017,2,2983448.628,9598,14812,8,Pass,80,Cross,"[{'y': 2, 'x': 82}, {'y': 100, 'x': 100}]","[{'id': 401}, {'id': 801}, {'id': 1802}]",49.724144
1460,263885613,2058017,2,2985869.275,4418,25381,4,Goalkeeper leaving line,40,Goalkeeper leaving line,"[{'y': 0, 'x': 0}, {'y': 98, 'x': 18}]",[],49.764488


In [200]:
# Convert Wyscout to SPADL Format Input (Events Dataframe, home_team_id) --> Rusia
spadl_events_wc_2018_df = convert_to_actions(event_wc_2018_df, 4418)
spadl_events_wc_2018_df

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id
0,2058017,1,1.892339,9598,14943,52.50,34.68,52.50,34.68,263883958,0,0,1,0
1,2058017,1,2.890857,9598,69968,52.50,34.68,64.05,35.36,NaN,0,21,1,1
2,2058017,1,3.889375,9598,69968,64.05,35.36,69.30,50.32,263883959,0,0,1,2
3,2058017,1,6.140946,9598,8287,69.30,50.32,73.50,63.24,263883960,0,0,1,3
4,2058017,1,9.226570,9598,69409,73.50,63.24,93.45,44.20,263883963,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,2058017,2,2972.985039,9598,69968,68.25,0.00,56.70,13.60,263885651,2,2,1,1128
1129,2058017,2,2978.301867,9598,3476,56.70,13.60,37.80,4.08,263885652,0,0,1,1129
1130,2058017,2,2980.875248,9598,14812,37.80,4.08,18.90,1.36,NaN,0,21,1,1130
1131,2058017,2,2983.448628,9598,14812,18.90,1.36,0.00,68.00,263885654,0,1,0,1131


In [201]:
# 1. convert actions to game states
home_team_id = 4418
gamestates = fs.gamestates(spadl_events_wc_2018_df, 3)
gamestates = fs.play_left_to_right(gamestates, home_team_id)
gamestates

[      game_id  period_id  time_seconds  team_id  player_id  start_x  start_y  \
 0     2058017          1      1.892339     9598      14943    52.50    33.32   
 1     2058017          1      2.890857     9598      69968    52.50    33.32   
 2     2058017          1      3.889375     9598      69968    40.95    32.64   
 3     2058017          1      6.140946     9598       8287    35.70    17.68   
 4     2058017          1      9.226570     9598      69409    31.50     4.76   
 ...       ...        ...           ...      ...        ...      ...      ...   
 1128  2058017          2   2972.985039     9598      69968    36.75    68.00   
 1129  2058017          2   2978.301867     9598       3476    48.30    54.40   
 1130  2058017          2   2980.875248     9598      14812    67.20    63.92   
 1131  2058017          2   2983.448628     9598      14812    86.10    66.64   
 1132  2058017          2   3002.148765     4418      25381    14.70    38.76   
 
        end_x  end_y origi

In [202]:
# 2. compute features
xfns = [
    fs.actiontype, 
    fs.result, 
    fs.bodypart, 
    fs.time, 
    fs.startlocation, 
    fs.endlocation,
    fs.startpolar,
    fs.endpolar,
    fs.movement,
    fs.team,
    fs.time_delta,
    fs.space_delta,
    # fs.goalscore
]
X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
print(X)

      type_id_a0  type_id_a1  type_id_a2  result_id_a0  result_id_a1  \
0              0           0           0             1             1   
1             21           0           0             1             1   
2              0          21           0             1             1   
3              0           0          21             1             1   
4              0           0           0             1             1   
...          ...         ...         ...           ...           ...   
1128           2          18           0             1             0   
1129           0           2          18             1             1   
1130          21           0           2             1             1   
1131           1          21           0             0             1   
1132           0           1          21             0             0   

      result_id_a2  bodypart_id_a0  bodypart_id_a1  bodypart_id_a2  \
0                1               0               0               

In [203]:
# 3. compute labels
# yfns = [lab.scores, lab.concedes]
# Y = pd.concat([fn(spadl_events_wc_2018_df) for fn in yfns], axis=1)